<a href="https://colab.research.google.com/github/RohanCoderiiitb/FFT-Processor/blob/main/FFT_Processor_Evaluation_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparission between FFT results for data in FP4 and FP8 format

We are comparing results obtained by using FP4(E2M1), FP4(E1M2), FP8 with those of FP32.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def quantize_fp_custom(x, exp_bits, mant_bits, bias):
    """
    Hardware-accurate quantization for custom FP formats.
    Supports saturation and flushing underflows to zero.
    """
    if x == 0: return 0.0
    sign = np.sign(x)
    val = abs(x)

    exp = np.floor(np.log2(val))

    max_exp = (2**exp_bits - 1) - bias
    if exp > max_exp:
        max_val = (2**max_exp) * (1 + (2**mant_bits - 1) / 2**mant_bits)
        return sign * max_val

    min_exp = 1 - bias
    if exp < min_exp:
        return 0.0

    mant = val / (2**exp) - 1.0
    mant_q = np.round(mant * (2**mant_bits)) / (2**mant_bits)

    return sign * (1 + mant_q) * (2**exp)

def quant_e2m1(x): return quantize_fp_custom(x, exp_bits=2, mant_bits=1, bias=1)
def quant_e1m2(x): return quantize_fp_custom(x, exp_bits=1, mant_bits=2, bias=0)
def quant_fp8(x):  return quantize_fp_custom(x, exp_bits=4, mant_bits=3, bias=7)

In [3]:
def fft_radix2_hw_accurate(x, quant_func):
    x = x.astype(np.complex64).copy()
    N = len(x)
    stages = int(np.log2(N))

    for s in range(stages):
        m = 2**(s+1)
        for k in range(0, N, m):
            for j in range(m//2):
                w_ideal = np.exp(-2j * np.pi * j / m)
                wr_q = quant_func(w_ideal.real)
                wi_q = quant_func(w_ideal.imag)
                W_q = complex(wr_q, wi_q)

                item = x[k+j+m//2]
                real_part = quant_func(item.real * W_q.real) - quant_func(item.imag * W_q.imag)
                imag_part = quant_func(item.real * W_q.imag) + quant_func(item.imag * W_q.real)
                t = complex(quant_func(real_part), quant_func(imag_part))

                a = x[k+j]

                x[k+j] = complex(quant_func(a.real + t.real), quant_func(a.imag + t.imag))
                x[k+j+m//2] = complex(quant_func(a.real - t.real), quant_func(a.imag - t.imag))

    return x

## Testing using the following signals:
1. Single tone sinusoid
2. Dual tone sinusoid
3. Strong + Weak Signal
4. White Gaussian Noise
5. Impulse

Signal to Quantization Noise Ratio(SQNR), Shape Error and Spectral Correlation Coefficient are the metrics used to analyse the performance of different floating point architectures

## Metrics

In [4]:
def compute_sqnr(ref, quant):
  signal_power = np.sum(np.abs(ref)**2)
  noise_power = np.sum(np.abs(ref - quant)**2)
  return 10*np.log10(signal_power/(noise_power + 1e-18))

def compute_mean_shape_error(ref, quant):
  mag_ref = np.abs(ref)
  mag_quant = np.abs(quant)
  p_ref = np.max(mag_ref)
  p_quant = np.max(mag_quant)
  if p_quant < 1e-12:
    shape_err = 1.0
  else:
    shape_err = np.mean(np.abs((mag_ref/p_ref) - (mag_quant/p_quant)))
  return shape_err

def compute_spectral_correlation_coefficient(ref, quant):
  mag_ref = np.abs(ref)
  mag_quant = np.abs(quant)
  p_ref = np.max(mag_ref)
  p_quant = np.max(mag_quant)
  if p_quant < 1e-12 or np.std(mag_quant)==0:
    corr = 0.0
  else:
    corr = np.corrcoef(mag_ref, mag_quant)[0,1]
  return corr

### Single Tone Sinusoid

In [5]:
N = 32
k = 3
n = np.arange(N)
x = np.sin(2*np.pi*k*n/N)

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x,quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x,quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x,quant_fp8)

In [6]:
import pandas as pd

data = [
    {"Signal Name": "Single Tone Sinusoid", "Data Format": "FP4 (E2M1)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e2m1),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e2m1),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e2m1)},

    {"Signal Name": "Single Tone Sinusoid", "Data Format": "FP4 (E1M2)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e1m2),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e1m2),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e1m2)},

    {"Signal Name": "Single Tone Sinusoid", "Data Format": "FP8 (E4M3)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp8_e4m3),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp8_e4m3),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp8_e4m3)},
]

results_table = pd.DataFrame(data)

results_table = results_table[["Signal Name", "Data Format", "SQNR (dB)", "Shape Error", "Correlation"]]

print("FFT Architecture Performance Comparison Table")
print("="*80)
display(results_table)

FFT Architecture Performance Comparison Table


,Signal Name,Data Format,SQNR (dB),Shape Error,Correlation
0,Single Tone Sinusoid,FP4 (E2M1),-0.100613,0.200444,-0.130297
1,Single Tone Sinusoid,FP4 (E1M2),0.000000,1.000000,0.000000
2,Single Tone Sinusoid,FP8 (E4M3),-2.505882,0.312159,-0.175921


### Dual Tone Sinusoid

In [7]:
x = (np.sin(2*np.pi*3*n/N) + 0.5*np.sin(2*np.pi*5*n/N))
x = (x / np.max(np.abs(x))) * 2.0

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x, quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x, quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x, quant_fp8)

In [8]:
import pandas as pd

data = [
    {"Signal Name": "Dual Tone Sinusoid", "Data Format": "FP4 (E2M1)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e2m1),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e2m1),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e2m1)},

    {"Signal Name": "Dual Tone Sinusoid", "Data Format": "FP4 (E1M2)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e1m2),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e1m2),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e1m2)},

    {"Signal Name": "Dual Tone Sinusoid", "Data Format": "FP8 (E4M3)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp8_e4m3),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp8_e4m3),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp8_e4m3)},
]

results_table = pd.DataFrame(data)

results_table = results_table[["Signal Name", "Data Format", "SQNR (dB)", "Shape Error", "Correlation"]]

print("FFT Architecture Performance Comparison Table")
print("="*80)
display(results_table)

FFT Architecture Performance Comparison Table


,Signal Name,Data Format,SQNR (dB),Shape Error,Correlation
0,Dual Tone Sinusoid,FP4 (E2M1),-1.119597,0.343967,-0.144972
1,Dual Tone Sinusoid,FP4 (E1M2),0.000000,1.000000,0.000000
2,Dual Tone Sinusoid,FP8 (E4M3),-2.417982,0.301023,-0.211203


### Strong + Weak Signal

In [9]:
x = (1.0 * np.sin(2*np.pi*3*n/N) + 0.1 * np.sin(2*np.pi*12*n/N))
x = (x / np.max(np.abs(x))) * 2.0

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x, quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x, quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x, quant_fp8)

In [10]:
import pandas as pd

data = [
    {"Signal Name": "Strong + Weak Signal", "Data Format": "FP4 (E2M1)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e2m1),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e2m1),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e2m1)},

    {"Signal Name": "Strong + Weak Signal", "Data Format": "FP4 (E1M2)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e1m2),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e1m2),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e1m2)},

    {"Signal Name": "Strong + Weak Signal", "Data Format": "FP8 (E4M3)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp8_e4m3),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp8_e4m3),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp8_e4m3)},
]

results_table = pd.DataFrame(data)

results_table = results_table[["Signal Name", "Data Format", "SQNR (dB)", "Shape Error", "Correlation"]]

print("FFT Architecture Performance Comparison Table")
print("="*80)
display(results_table)

FFT Architecture Performance Comparison Table


,Signal Name,Data Format,SQNR (dB),Shape Error,Correlation
0,Strong + Weak Signal,FP4 (E2M1),-0.858604,0.467267,-0.184547
1,Strong + Weak Signal,FP4 (E1M2),0.000000,1.000000,0.000000
2,Strong + Weak Signal,FP8 (E4M3),-2.394800,0.310523,-0.060413


### White Gaussian Noise

In [11]:
np.random.seed(42)
x = np.random.normal(0, 0.5, N)
x = (x / np.max(np.abs(x))) * 2.0

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x, quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x, quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x, quant_fp8)

In [12]:
import pandas as pd

data = [
    {"Signal Name": "White Gaussian Noise", "Data Format": "FP4 (E2M1)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e2m1),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e2m1),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e2m1)},

    {"Signal Name": "White Gaussian Noise", "Data Format": "FP4 (E1M2)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e1m2),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e1m2),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e1m2)},

    {"Signal Name": "White Gaussian Noise", "Data Format": "FP8 (E4M3)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp8_e4m3),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp8_e4m3),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp8_e4m3)},
]

results_table = pd.DataFrame(data)

results_table = results_table[["Signal Name", "Data Format", "SQNR (dB)", "Shape Error", "Correlation"]]

print("FFT Architecture Performance Comparison Table")
print("="*80)
display(results_table)

FFT Architecture Performance Comparison Table


,Signal Name,Data Format,SQNR (dB),Shape Error,Correlation
0,White Gaussian Noise,FP4 (E2M1),-0.887995,0.403050,-0.344572
1,White Gaussian Noise,FP4 (E1M2),0.000000,1.000000,0.000000
2,White Gaussian Noise,FP8 (E4M3),-2.176097,0.306711,-0.054752


### Impulse Signal

In [13]:
x = np.zeros(N)
x[0] = 2.0

X_fft_ref = np.fft.fft(x)

X_fp4_e2m1 = fft_radix2_hw_accurate(x, quant_e2m1)
X_fp4_e1m2 = fft_radix2_hw_accurate(x, quant_e1m2)
X_fp8_e4m3 = fft_radix2_hw_accurate(x, quant_fp8)

In [14]:
import pandas as pd

data = [
    {"Signal Name": "Impulse", "Data Format": "FP4 (E2M1)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e2m1),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e2m1),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e2m1)},

    {"Signal Name": "Impulse", "Data Format": "FP4 (E1M2)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp4_e1m2),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp4_e1m2),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp4_e1m2)},

    {"Signal Name": "Impulse", "Data Format": "FP8 (E4M3)",
     "SQNR (dB)": compute_sqnr(X_fft_ref, X_fp8_e4m3),
     "Shape Error": compute_mean_shape_error(X_fft_ref, X_fp8_e4m3),
     "Correlation": compute_spectral_correlation_coefficient(X_fft_ref, X_fp8_e4m3)},
]

results_table = pd.DataFrame(data)

results_table = results_table[["Signal Name", "Data Format", "SQNR (dB)", "Shape Error", "Correlation"]]

print("FFT Architecture Performance Comparison Table")
print("="*80)
display(results_table)

FFT Architecture Performance Comparison Table


,Signal Name,Data Format,SQNR (dB),Shape Error,Correlation
0,Impulse,FP4 (E2M1),201.0721,0.0,0.0
1,Impulse,FP4 (E1M2),201.0721,0.0,0.0
2,Impulse,FP8 (E4M3),201.0721,0.0,0.0
